# Pydantic AI Walkthrough

Inspired bij: https://ai.pydantic.dev/agents/


Setup the environment


In [9]:
import dotenv
import os
import logfire

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")
logfire_token = os.getenv("LOGFIRE_TOKEN")

# configure logfire
logfire.configure(token=logfire_token)
logfire.instrument_pydantic_ai()

1 Definieer een basis agent


In [10]:
from pydantic_ai import Agent
from pydantic_ai.models.anthropic import AnthropicModel

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = AnthropicModel(os.getenv("DEFAULT_MODEL"))

basic_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

Stel een vraag aan de agent


In [11]:
result = basic_agent.run_sync("geef een defintie van een agent")
print(result.output)

11:31:20.795 basic_agent run
11:31:20.795   chat claude-3-5-haiku-20241022


Logfire project URL: ]8;id=386335;https://logfire-eu.pydantic.dev/elbrink/claude-test\https://logfire-eu.pydantic.dev/elbrink/claude-test]8;;\

Hier is een definitie van een agent in de context van informatica en kunstmatige intelligentie:

Een agent is een computerprogramma of systeem dat:

1. Autonoom kan waarnemen
2. Beslissingen kan nemen
3. Acties kan uitvoeren
4. Gericht is op het bereiken van specifieke doelen
5. Kan leren en zich kan aanpassen aan veranderende omstandigheden

Belangrijke kenmerken van een agent zijn:
- Autonomie
- Reactievermogen
- Pro-activiteit
- Sociale vaardigheid
- Intelligentie

Agents kunnen voorkomen in verschillende vormen, zoals:
- Softwareagents
- Intelligente robots
- Virtuele assistenten
- Computerspel-AI
- Beslissingsondersteunende systemen

Het primaire doel van een agent is om effectief te functioneren in een bepaalde omgeving door middel van waarneming, redenering en handelen.


Definieer tools voor de agent


In [12]:
tool_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)


@tool_agent.tool_plain
def tool_bereken_btw(bedrag: float, btw_percentage: float):
    """
    Bereken BTW over een bedrag.
    """
    return tf.calculate_btw(bedrag, btw_percentage)


@tool_agent.tool_plain
def tool_bereken_korting(bedrag: float, korting_percentage: float):
    """
    Bereken korting over een bedrag.
    """
    return tf.calculate_discount(bedrag, korting_percentage)


@tool_agent.tool_plain
def tool_get_inwoners_gemeente(gemeente_naam: str, jaar: str):
    """
    Haal inwonersaantallen op van Nederlandse gemeenten via CBS API.
    """
    return tf.get_inwoners_gemeente(gemeente_naam, jaar)

Test een aantal tool calls


In [13]:
result2 = tool_agent.run_sync(
    "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld"
)
print(result2.output)

11:31:26.919 tool_agent run
11:31:26.920   chat claude-3-5-haiku-20241022
11:31:30.689   running 1 tool
11:31:30.689     running tool: tool_bereken_btw
11:31:30.690   chat claude-3-5-haiku-20241022
11:31:34.940   running 1 tool
11:31:34.940     running tool: tool_bereken_korting
11:31:34.941   chat claude-3-5-haiku-20241022
Hier is een gedetailleerd overzicht:

1. Oorspronkelijke prijs exclusief BTW: €100,00
2. BTW (21%): €21,00
3. Prijs inclusief BTW: €121,00
4. Korting (15%): €18,15
5. Uiteindelijke te betalen prijs: €102,85

Samenvattend:
- Je begint met een product van €100,00
- Er wordt 21% BTW toegevoegd, wat het bedrag brengt op €121,00
- Vervolgens krijg je 15% korting, wat €18,15 korting oplevert
- Je eindigt met een te betalen bedrag van €102,85

De korting is dus rechtstreeks van toepassing op het totale bedrag inclusief BTW, wat resulteert in een aanzienlijke besparing voor jou.


In [14]:
result = tool_agent.run_sync(
    "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024. Laat ook het aantal inwoners van Groningen zien."
)
print(result.output)

11:31:39.874 tool_agent run
11:31:39.875   chat claude-3-5-haiku-20241022
11:31:47.566   running 7 tools
11:31:47.567     running tool: tool_get_inwoners_gemeente
11:31:47.567     running tool: tool_get_inwoners_gemeente
11:31:47.567     running tool: tool_get_inwoners_gemeente
11:31:47.567     running tool: tool_get_inwoners_gemeente
11:31:47.568     running tool: tool_get_inwoners_gemeente
11:31:47.568     running tool: tool_get_inwoners_gemeente
11:31:47.568     running tool: tool_get_inwoners_gemeente
11:31:48.714   chat claude-3-5-haiku-20241022
Op basis van de resultaten kan ik je het volgende overzicht geven van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2024:

1. Amsterdam: 931.298 inwoners
2. Rotterdam: 670.610 inwoners
3. Utrecht: 374.238 inwoners
4. Eindhoven: 246.417 inwoners
5. Groningen: 243.768 inwoners
6. Tilburg: 229.836 inwoners

Opmerking: Voor Den Haag kon ik helaas geen gegevens ophalen, dus deze gemeente ontbreekt in de lijst. Normaal gesp

In [15]:
result = tool_agent.run_sync(
    "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan."
)
print(result.output)

11:31:54.133 tool_agent run
11:31:54.133   chat claude-3-5-haiku-20241022
11:31:57.346   running 1 tool
11:31:57.347     running tool: tool_get_inwoners_gemeente
11:31:58.167   chat claude-3-5-haiku-20241022
Op basis van de gegevens van het CBS zal Eindhoven in 2025 naar verwachting 249.054 inwoners hebben. 

Als elke inwoner 15 euro zou betalen, zou dat neerkomen op:
249.054 × €15 = €3.735.810

Dus als alle inwoners van Eindhoven in 2025 15 euro zouden betalen, zou het totaalbedrag €3.735.810 zijn. 

Echter, ik wil wel benadrukken dat het niet gebruikelijk of wenselijk is om zomaar geld van alle inwoners te vorderen zonder een duidelijke reden of tegenprestatie. Als u specifieke plannen of doelen heeft, zou ik graag meer willen weten over de context.


Voeg geheugen toe aan de agent


In [16]:
mem_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
)

result = mem_agent.run_sync(
    "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor.",
    message_history=result2.all_messages(),
)
print(result.output)

11:32:03.421 mem_agent run
11:32:03.422   chat claude-3-5-haiku-20241022
Laat me dat even voor je opzoeken in de vorige conversatie:

BTW-bedrag: €21,00
Korting: €18,15

Dit zijn de bedragen die ik in mijn vorige berekening heb vastgesteld voor product X.


PyDantic MCP client


In [17]:
import asyncio
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

server = MCPServerSSE(url="http://localhost:8050/sse")
mcp_agent = Agent(
    model=model,
    output_type=str,
    system_prompt=(
        "je bent een agent die op alle vragen een verstandig antwoord geeft"
    ),
    mcp_servers=[server],
)

In [18]:
async def run_query():
    async with mcp_agent.run_mcp_servers():
        print(await server.list_tools())
        result = await mcp_agent.run(
            "Geef een overzicht van de top-10 gemeenten in Nederland met het grootste aantal inwoners in 2024."
        )
        print(result.output)
        result = await mcp_agent.run(
            "Hoeveel inwoners had Rotterdam meer dan Eindhoven in 2024?"
        )
        print(result.output)


asyncio.run(run_query())

[ToolDefinition(name='calculate_btw', description='\n    Bereken BTW over een bedrag.\n\n    Args:\n        bedrag: Het bedrag exclusief BTW\n        btw_percentage: BTW percentage (standaard 21%)\n\n    Returns:\n        Dict met BTW berekening resultaten\n    ', parameters_json_schema={'properties': {'bedrag': {'title': 'Bedrag', 'type': 'number'}, 'btw_percentage': {'default': 21.0, 'title': 'Btw Percentage', 'type': 'number'}}, 'required': ['bedrag'], 'title': 'calculate_btwArguments', 'type': 'object'}), ToolDefinition(name='calculate_discount', description='\n    Bereken korting over een bedrag.\n    ', parameters_json_schema={'properties': {'bedrag': {'title': 'Bedrag', 'type': 'number'}, 'korting_percentage': {'title': 'Korting Percentage', 'type': 'number'}}, 'required': ['bedrag', 'korting_percentage'], 'title': 'calculate_discountArguments', 'type': 'object'}), ToolDefinition(name='get_inwoners_gemeente', description='\n    Geef het aantal inwoners van een gemeente.\n    ', 

11:32:09.127   running 1 tool
11:32:09.128     running tool: get_inwoners_gemeente
11:32:10.030   chat claude-3-5-haiku-20241022
11:32:12.794   running 1 tool
11:32:12.795     running tool: get_inwoners_gemeente
11:32:13.563   chat claude-3-5-haiku-20241022
11:32:16.195   running 1 tool
11:32:16.196     running tool: get_inwoners_gemeente
11:32:17.074   chat claude-3-5-haiku-20241022
11:32:20.413   running 1 tool
11:32:20.413     running tool: get_inwoners_gemeente
11:32:21.086   chat claude-3-5-haiku-20241022
11:32:23.500   running 1 tool
11:32:23.500     running tool: get_inwoners_gemeente
11:32:24.316   chat claude-3-5-haiku-20241022
11:32:27.162   running 1 tool
11:32:27.163     running tool: get_inwoners_gemeente
11:32:28.149   chat claude-3-5-haiku-20241022
11:32:30.496   running 1 tool
11:32:30.497     running tool: get_inwoners_gemeente
11:32:31.430   chat claude-3-5-haiku-20241022
11:32:35.465   running 1 tool
11:32:35.466     running tool: get_inwoners_gemeente
11:32:36.085  